# Aufgabe3-3

## 0) Verbindungsaufbau und Bibliotheken

In [33]:
import oracledb
import random
from datetime import datetime, timedelta
import pandas as pd

oracledb.init_oracle_client(lib_dir=r"instantclient_19_11")

username = ""
password = ""
dsn = "oracle19.cs.univie.ac.at:1521/orclcdb"

connection = oracledb.connect(
    user=username,
    password=password,
    dsn=dsn,
)
cursor = connection.cursor()

print("Connected to Oracle DB successfully!")

Connected to Oracle DB successfully!


## 1) Vorherige Tabellen löschen

Wir droppen alle Tabellen, falls sie existieren, um Konflikte zu vermeiden.

In [35]:
tables_to_drop = ["Carries", "Flight", "Airline", "Passenger"]
for tbl in tables_to_drop:
    drop_sql = f"DROP TABLE {tbl} CASCADE CONSTRAINTS"
    try:
        cursor.execute(drop_sql)
        connection.commit()
        print(f"Dropped table {tbl}")
    except oracledb.Error as e:
        print(f"Skipping drop for {tbl}, possibly doesn't exist: {e}")

Dropped table Carries
Dropped table Flight
Dropped table Airline
Dropped table Passenger


## 2) Tabellen neu erstellen

Wir legen `Airline`, `Flight`, `Passenger` und `Carries` an.

In [37]:
create_airline = """
CREATE TABLE Airline (
    airlineID      NUMBER PRIMARY KEY,
    Name           VARCHAR2(100),
    Headquarters   VARCHAR2(200)
)
"""
cursor.execute(create_airline)

create_flight = """
CREATE TABLE Flight (
    flightNumber   VARCHAR2(10) PRIMARY KEY,
    DepartureTime  TIMESTAMP,
    ArrivalTime    TIMESTAMP,
    airlineID      NUMBER,
    CONSTRAINT fk_flight_airline
        FOREIGN KEY (airlineID)
        REFERENCES Airline(airlineID)
        ON DELETE CASCADE
)
"""
cursor.execute(create_flight)

create_passenger = """
CREATE TABLE Passenger (
    passengerID    NUMBER PRIMARY KEY,
    FirstName      VARCHAR2(100),
    Surname        VARCHAR2(100),
    PassportNumber VARCHAR2(50)
)
"""
cursor.execute(create_passenger)

create_carries = """
CREATE TABLE Carries (
    flightNumber   VARCHAR2(10),
    passengerID    NUMBER,
    CONSTRAINT pk_carries
        PRIMARY KEY (flightNumber, passengerID),
    CONSTRAINT fk_carries_flight
        FOREIGN KEY (flightNumber) 
        REFERENCES Flight(flightNumber)
        ON DELETE CASCADE,
    CONSTRAINT fk_carries_passenger
        FOREIGN KEY (passengerID)
        REFERENCES Passenger(passengerID)
        ON DELETE CASCADE
)
"""
cursor.execute(create_carries)

connection.commit()
print("All tables created successfully!")

All tables created successfully!


## 3) Einfügen großer Datenmengen

- 500 Airlines
- 3000 Passengers
- 2000 Flights
- Verknüpfung über Carries


In [39]:
# (3.1) Einfügen von 500 Airlines
for i in range(1, 501):
    sql = """
        INSERT INTO Airline (airlineID, Name, Headquarters)
        VALUES (:1, :2, :3)
    """
    airline_id = i
    name = f"Airline_{i}"
    headquarters = f"Headquarters_{i}"
    cursor.execute(sql, [airline_id, name, headquarters])

# (3.2) Einfügen von 3000 Passengers
for i in range(1, 3001):
    sql = """
        INSERT INTO Passenger (passengerID, FirstName, Surname, PassportNumber)
        VALUES (:1, :2, :3, :4)
    """
    passenger_id = i
    first_name = f"FName{i}"
    surname = f"LName{i}"
    passport = f"P{i:05d}"
    cursor.execute(sql, [passenger_id, first_name, surname, passport])

# (3.3) Einfügen von 2000 Flights
for i in range(1, 2001):
    flight_num = f"FN{i}"
    hour_offset = i % 24
    chosen_airline = random.randint(1, 500)

    base_dep = datetime(2025, 1, 1, 8, 0, 0)
    base_arr = datetime(2025, 1, 1, 12, 0, 0)
    departure_time = base_dep + timedelta(hours=hour_offset)
    arrival_time = base_arr + timedelta(hours=hour_offset)
    
    sql = """
    INSERT INTO Flight (
        flightNumber, DepartureTime, ArrivalTime, airlineID
    )
    VALUES (:1, :2, :3, :4)
    """
    cursor.execute(sql, (
        flight_num, 
        departure_time,
        arrival_time, 
        chosen_airline
    ))

# (3.4) Verknüpfung mit Carries: jeder Passenger nimmt 1-4 Zufallsflüge.
for passenger_id in range(1, 3001):
    flight_count = random.randint(1, 4)  # bis zu 4 Flüge
    flight_ids = random.sample(range(1, 2001), flight_count)
    for fid in flight_ids:
        flight_num = f"FN{fid}"
        sql = "INSERT INTO Carries (flightNumber, passengerID) VALUES (:1, :2)"
        cursor.execute(sql, (flight_num, passenger_id))

connection.commit()
print("Inserted 500 Airlines, 3000 Passengers, 2000 Flights. Carries data inserted. Large dataset created!")

Inserted 500 Airlines, 3000 Passengers, 2000 Flights. Carries data inserted. Large dataset created!


### Überprüfung der Anzahl Datensätze

In [41]:
tables = ["Airline", "Passenger", "Flight", "Carries"]
for tbl in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {tbl}")
    count = cursor.fetchone()[0]
    print(f"{tbl} has {count} rows.")

Airline has 500 rows.
Passenger has 3000 rows.
Flight has 2000 rows.
Carries has 7482 rows.


## 4) Erweiterte Queries

Jede Abfrage nutzt mindestens zwei fortgeschrittene SQL-Konstrukte (GROUP BY, HAVING, ALL, Subselect).

Hilfsfunktion, um SQL-Abfragen schön darzustellen

In [43]:
def run_query_and_show_dataframe(sql_query, description):
    print(f"### {description}")
    print(f"SQL:\n{sql_query}\n")

    cursor.execute(sql_query)
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    df = pd.DataFrame(rows, columns=columns)
    display(df)

### 4.1) Query #1: GROUP BY + HAVING
Zeige, wie viele Flüge jede Airline hat, aber nur Airlines mit mehr als 8 Flügen.

In [45]:
query_4_1 = """
SELECT
    A.airlineID,
    A.Name AS AirlineName,
    COUNT(F.flightNumber) AS FlightCount
FROM Airline A
JOIN Flight F ON A.airlineID = F.airlineID
GROUP BY A.airlineID, A.Name
HAVING COUNT(F.flightNumber) > 8
"""

run_query_and_show_dataframe(
    sql_query=query_4_1,
    description="QUERY 4.1 (GROUP BY + HAVING, now showing airlineID)"
)

### QUERY 4.1 (GROUP BY + HAVING, now showing airlineID)
SQL:

SELECT
    A.airlineID,
    A.Name AS AirlineName,
    COUNT(F.flightNumber) AS FlightCount
FROM Airline A
JOIN Flight F ON A.airlineID = F.airlineID
GROUP BY A.airlineID, A.Name
HAVING COUNT(F.flightNumber) > 8




,AIRLINEID,AIRLINENAME,FLIGHTCOUNT
0,461,Airline_461,10
1,433,Airline_433,9
2,246,Airline_246,9
3,151,Airline_151,13
4,179,Airline_179,9
5,446,Airline_446,9
6,464,Airline_464,10
7,316,Airline_316,9
8,67,Airline_67,10
9,9,Airline_9,10


### 4.2) Query #2: Subselect + ALL
Finde Passagiere, deren ID größer ist als alle Passagier-IDs, die auf Flug 'FN100' unterwegs sind.

In [47]:
query_4_2 = """
SELECT
    p.passengerID,
    p.FirstName,
    p.Surname,
    (SELECT MAX(c2.passengerID) 
       FROM Carries c2 
       WHERE c2.flightNumber = 'FN100'
    ) AS max_passengerID_on_FN100
FROM Passenger p
WHERE p.passengerID > ALL (
    SELECT c.passengerID
    FROM Carries c
    WHERE c.flightNumber = 'FN100'
)
"""

run_query_and_show_dataframe(
    sql_query=query_4_2,
    description="QUERY 4.2 (Subselect + ALL) with visible max passengerID from FN100"
)

### QUERY 4.2 (Subselect + ALL) with visible max passengerID from FN100
SQL:

SELECT
    p.passengerID,
    p.FirstName,
    p.Surname,
    (SELECT MAX(c2.passengerID) 
       FROM Carries c2 
       WHERE c2.flightNumber = 'FN100'
    ) AS max_passengerID_on_FN100
FROM Passenger p
WHERE p.passengerID > ALL (
    SELECT c.passengerID
    FROM Carries c
    WHERE c.flightNumber = 'FN100'
)




,PASSENGERID,FIRSTNAME,SURNAME,MAX_PASSENGERID_ON_FN100
0,2108,FName2108,LName2108,2107
1,2109,FName2109,LName2109,2107
2,2110,FName2110,LName2110,2107
3,2111,FName2111,LName2111,2107
4,2112,FName2112,LName2112,2107
...,...,...,...,...
888,2996,FName2996,LName2996,2107
889,2997,FName2997,LName2997,2107
890,2998,FName2998,LName2998,2107
891,2999,FName2999,LName2999,2107


### 4.3) Query #3: GROUP BY + Subselect
Zeige Passagiere, die mehr Flüge als der Durchschnitt aller Passagiere haben.

In [49]:
query_4_3 = """
WITH overall AS (
    SELECT AVG(cnt) AS avg_cnt
    FROM (
        SELECT passengerID, COUNT(*) AS cnt
        FROM Carries
        GROUP BY passengerID
    )
)
SELECT 
    c.passengerID,
    COUNT(c.flightNumber) AS total_flights,
    o.avg_cnt AS overall_avg
FROM Carries c
CROSS JOIN overall o
GROUP BY c.passengerID, o.avg_cnt
HAVING COUNT(c.flightNumber) > o.avg_cnt
"""

run_query_and_show_dataframe(
    sql_query=query_4_3,
    description="QUERY 4.3 (GROUP BY + Subselect) showing each passenger's total flights vs. overall average"
)

### QUERY 4.3 (GROUP BY + Subselect) showing each passenger's total flights vs. overall average
SQL:

WITH overall AS (
    SELECT AVG(cnt) AS avg_cnt
    FROM (
        SELECT passengerID, COUNT(*) AS cnt
        FROM Carries
        GROUP BY passengerID
    )
)
SELECT 
    c.passengerID,
    COUNT(c.flightNumber) AS total_flights,
    o.avg_cnt AS overall_avg
FROM Carries c
CROSS JOIN overall o
GROUP BY c.passengerID, o.avg_cnt
HAVING COUNT(c.flightNumber) > o.avg_cnt




,PASSENGERID,TOTAL_FLIGHTS,OVERALL_AVG
0,790,4,2.494
1,796,3,2.494
2,820,3,2.494
3,842,4,2.494
4,872,3,2.494
...,...,...,...
1488,2935,3,2.494
1489,2939,3,2.494
1490,2965,4,2.494
1491,2966,3,2.494


## 5) Stored Procedures (Update & Delete)

Wir erstellen zwei Prozeduren:
- `update_passenger_surname`  
- `delete_flights_with_prefix`

Danach werden sie aufgerufen und überprüft.

In [51]:
sp_update = """
CREATE OR REPLACE PROCEDURE update_passenger_surname(
    p_passengerID IN NUMBER,
    p_newSurname  IN VARCHAR2
) AS
BEGIN
    UPDATE Passenger
    SET Surname = p_newSurname
    WHERE passengerID = p_passengerID;
    COMMIT;
END;
"""

cursor.execute(sp_update)
print("Procedure update_passenger_surname created.")

Procedure update_passenger_surname created.


In [53]:
sp_delete = """
CREATE OR REPLACE PROCEDURE delete_flights_with_prefix(
    p_airlineID     IN NUMBER,
    p_flightPrefix  IN VARCHAR2
) AS
BEGIN
    DELETE FROM Flight
    WHERE airlineID = p_airlineID
      AND flightNumber LIKE p_flightPrefix || '%';
    COMMIT;
END;
"""

cursor.execute(sp_delete)
print("Procedure delete_flights_with_prefix created.")

Procedure delete_flights_with_prefix created.


Aufruf der Stored Procedures

In [55]:
# Beispiel 1: Update
update_call = """
BEGIN
    update_passenger_surname(:1, :2);
END;
"""
cursor.execute(update_call, [10, "NEWLNAME"])
connection.commit()
print("Called update_passenger_surname for passengerID=10.")

# Beispiel 2: Delete
delete_call = """
BEGIN
    delete_flights_with_prefix(:airlineID, :prefix);
END;
"""
cursor.execute(delete_call, airlineID=20, prefix="FN1")
connection.commit()
print("Called delete_flights_with_prefix for airlineID=20, prefix='FN1'.")

Called update_passenger_surname for passengerID=10.
Called delete_flights_with_prefix for airlineID=20, prefix='FN1'.


Kurze Überprüfung

In [57]:
check_updated_sql = """
SELECT passengerID, FirstName, Surname
FROM Passenger
WHERE passengerID = 10
"""

cursor.execute(check_updated_sql)
row = cursor.fetchone()
print("Verification of update_passenger_surname:", row)

Verification of update_passenger_surname: (10, 'FName10', 'NEWLNAME')


In [59]:
final_tables = ["Airline", "Passenger", "Flight", "Carries"]
for tbl in final_tables:
    cursor.execute(f"SELECT COUNT(*) FROM {tbl}")
    cnt = cursor.fetchone()[0]
    print(f"Table {tbl} final count: {cnt}")

Table Airline final count: 500
Table Passenger final count: 3000
Table Flight final count: 1996
Table Carries final count: 7470


## 6) Indexing & Query Optimization

Wir erstellen eine große Tabelle `kunden` mit 100.000 Einträgen, führen eine Abfrage vor und nach dem Erstellen eines Indexes durch und vergleichen die Ausführungspläne.

Kundentabelle droppen, falls vorhanden, dann neu anlegen.

In [ ]:
try:
    cursor.execute("DROP TABLE kunden CASCADE CONSTRAINTS")
    connection.commit()
    print("Dropped existing table kunden.")
except oracledb.Error as e:
    print("No existing table kunden. Proceeding with creation.")

create_kunden_table = """
CREATE TABLE kunden (
    id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY PRIMARY KEY,
    vorname    VARCHAR2(50),
    nachname   VARCHAR2(50),
    email      VARCHAR2(100),
    geburtsdatum DATE,
    stadt      VARCHAR2(50)
)
"""
cursor.execute(create_kunden_table)
connection.commit()
print("Created table kunden.")

No existing table kunden. Proceeding with creation.
Created table kunden.


Einfügen von 100.000 Einträgen in kunden

In [ ]:
batch_size = 10000
total_rows = 100000

insert_sql = """
INSERT INTO kunden (vorname, nachname, email, geburtsdatum, stadt)
VALUES (:vorname, :nachname, :email, :geburtsdatum, :stadt)
"""

for i in range(1, total_rows + 1):
    vorname = f"Vorname{i}"
    nachname = f"Nachname{i}"
    email = f"email{i}@example.com"
    geburtsdatum = datetime.date(1990, 1, 1)
    stadt = "Wien" if random.random() < 0.5 else "Graz"

    cursor.execute(insert_sql, [vorname, nachname, email, geburtsdatum, stadt])

    if i % batch_size == 0:
        connection.commit()
        print(f"{i} rows inserted...")

connection.commit()
print(f"Finished inserting {total_rows} rows into kunden.")

10000 rows inserted...
20000 rows inserted...
30000 rows inserted...
40000 rows inserted...
50000 rows inserted...
60000 rows inserted...
70000 rows inserted...
80000 rows inserted...
90000 rows inserted...
100000 rows inserted...
Finished inserting 100000 rows into kunden.


Kontrolle der Zeilenzahl

In [61]:
cursor.execute("SELECT COUNT(*) FROM kunden")
count_kunden = cursor.fetchone()[0]
print(f"'kunden' now has {count_kunden} rows.")

'kunden' now has 100000 rows.


EXPLAIN PLAN vor dem Index

In [63]:
explain_before_sql = """
EXPLAIN PLAN FOR
SELECT * 
FROM kunden
WHERE stadt = 'Wien'
"""
cursor.execute(explain_before_sql)

cursor.execute("SELECT PLAN_TABLE_OUTPUT FROM TABLE(DBMS_XPLAN.DISPLAY())")
plan_output = cursor.fetchall()

print("=== EXPLAIN PLAN (BEFORE INDEX) ===")
for line in plan_output:
    print(line[0])

=== EXPLAIN PLAN (BEFORE INDEX) ===
Plan hash value: 2881942826
 
----------------------------------------------------------------------------
| Id  | Operation         | Name   | Rows  | Bytes | Cost (%CPU)| Time     |
----------------------------------------------------------------------------
|   0 | SELECT STATEMENT  |        | 10000 |   625K|   102   (0)| 00:00:01 |
|*  1 |  TABLE ACCESS FULL| KUNDEN | 10000 |   625K|   102   (0)| 00:00:01 |
----------------------------------------------------------------------------
 
Predicate Information (identified by operation id):
---------------------------------------------------
 
   1 - filter("STADT"='Wien')


### Index erstellen auf Spalte `stadt`

In [65]:
try:
    cursor.execute("CREATE INDEX idx_stadt ON kunden(stadt)")
    connection.commit()
    print("Index idx_stadt created on kunden(stadt).")
except oracledb.Error as e:
    print(f"Could not create index: {e}")

Could not create index: ORA-00955: name is already used by an existing object


EXPLAIN PLAN nach dem Index

In [67]:
explain_after_sql = """
EXPLAIN PLAN FOR
SELECT * 
FROM kunden
WHERE stadt = 'Wien'
"""
cursor.execute(explain_after_sql)

cursor.execute("SELECT PLAN_TABLE_OUTPUT FROM TABLE(DBMS_XPLAN.DISPLAY())")
plan_output = cursor.fetchall()

print("=== EXPLAIN PLAN (AFTER INDEX) ===")
for line in plan_output:
    print(line[0])


=== EXPLAIN PLAN (AFTER INDEX) ===
Plan hash value: 2881942826
 
----------------------------------------------------------------------------
| Id  | Operation         | Name   | Rows  | Bytes | Cost (%CPU)| Time     |
----------------------------------------------------------------------------
|   0 | SELECT STATEMENT  |        | 10000 |   625K|   102   (0)| 00:00:01 |
|*  1 |  TABLE ACCESS FULL| KUNDEN | 10000 |   625K|   102   (0)| 00:00:01 |
----------------------------------------------------------------------------
 
Predicate Information (identified by operation id):
---------------------------------------------------
 
   1 - filter("STADT"='Wien')


## 8) Aufräumen

In [69]:
cursor.close()
connection.close()
print("Closed Oracle connection. Done!")

Closed Oracle connection. Done!
